# Minimizing the Loss over Training Data

When you have training data for classification, the loss is calculated based on the performance of the classifier over a batch of the training data.

The parameters of the neural network (the weights and biases) which minimize the loss are discovered by descending the loss gradient, as you can see below.

We've provided a utility class 'Data' (in data_reader.py) to load the training data (it works for all the toy problems).

In [1]:
import torch
import torch.nn.functional as F
from data_reader import Data

data = Data("data/toy_problem_1_train.txt")

labels, features = data.get_sample()

print("Labels:\n"+str(labels))

print("Features:\n"+str(features))
    
target = torch.autograd.Variable(torch.LongTensor(labels))
#print("Labels Tensor:\n"+str(target))

features = torch.autograd.Variable(torch.Tensor(features))
#print("Features Tensor:\n"+str(features))

Labels:
[0, 0, 1, 0, 0, 0, 1, 0, 1, 1]
Features:
[[-12, -79], [97, 27], [-45, 84], [4, -71], [81, 10], [-35, -36], [-84, 21], [79, -12], [24, 38], [-98, 78]]


We initialize the weights randomly.

We can now perform 100 learning iterations below as many times as we want.

Each learning iteration involves a forward pass and a backward pass.

The forward pass involves the computation of the loss from the training data and the current parameters.

The backward pass is performed automatically by Pytorch when you call loss.backward().

Pytorch calculates all the gradients with respect to the loss.

These gradients are stored in each parameter's 'grad' member variable.

Notice that the code for the learning iterations is almost identical to that of exercise 510.

In [3]:
weights = torch.nn.Parameter(torch.rand(2, 2))

print(weights)

for i in range(101):
    
    if weights.grad is not None:
        weights.grad.data.zero_()

    # Forward pass
    
    labels, features = data.get_sample()
    
    features = torch.autograd.Variable(torch.Tensor(features))
    #print(data)
    
    target = torch.autograd.Variable(torch.LongTensor(labels))
    #print(target)
    
    result = torch.mm(features, weights)
    #print(result)
    
    loss = F.cross_entropy(result, target)
    #print("Cross entropy loss: "+str(loss))
    
    # Backward pass
    
    loss.backward()
    
    gradient = weights.grad
    
    learning_rate = 0.01
    
    weights.data = weights.data - learning_rate * gradient.data
    
    if i % 10 == 0:
        print("The loss is now "+str(loss.data.item()))
    
print("\tThe weights are now "+str(weights.data))

torch.save(weights, "models/toy_problem_1_trained_model.bin")

Parameter containing:
tensor([[ 0.6774,  0.9258],
        [ 0.7395,  0.8862]])
The loss is now 10.548181533813477
The loss is now 1.5773188351886347e-05
The loss is now 0.0036009151954203844
The loss is now 0.010963144712150097
The loss is now 0.000298842613119632
The loss is now 9.246605969792654e-08
The loss is now 0.007362870965152979
The loss is now 9.522382242721505e-06
The loss is now 5.337378983227836e-09
The loss is now 0.1943223923444748
The loss is now 0.07547501474618912
	The weights are now tensor([[ 1.2251,  0.3782],
        [ 0.3570,  1.2688]])


## Parameters

As we know, the final parameters learnt by the algorithm should look something like this

$$\begin{bmatrix}1 & 0 \\ 0 & 1\end{bmatrix}$$

or this

$$\begin{bmatrix}2 & 1 \\ 1 & 2\end{bmatrix}$$

Basically the weights values at 0,0 and 1,1 in the matrix should be higher than the weights at 1,0 and 0,1.

## Classifier for Toy Problem 1

We have just trained a classifier for Toy Problem 1.

You can evaluate the performance of the classifier on the test data.

In [4]:
data = Data("data/toy_problem_1_test.txt")

weights = torch.load("models/toy_problem_1_trained_model.bin")

print(weights)

labels, features = data.get_all()

features = torch.autograd.Variable(torch.Tensor(features))
#print(features)

target = torch.autograd.Variable(torch.LongTensor(labels))
#print(target)

result = torch.mm(features, weights)
#print(result)

maxv, observed = torch.max(result, 1)

total = 0
correct = 0
for i in range(len(labels)):
    total += 1
    #print(str(target.data[i]) + " " + str(observed.data[i]))
    if target.data[i] == observed.data[i]:
        correct += 1
accuracy = correct / total
print("Accuracy: "+str(accuracy))

tensor([[ 1.2251,  0.3782],
        [ 0.3570,  1.2688]])
Accuracy: 0.985
